In [ ]:
# +METO+

In [ ]:
#pip install --upgrade openai
#pip install PyMuPDF
#pip install langchain

In [ ]:
#funciona bien solo que no esta separando solo a textos importantes
import subprocess
from pathlib import Path
import fitz  # PyMuPDF
import re

pdf_path = Path("C:\\meto\\mi documento.pdf")

# Comprobar si el PDF existe
if pdf_path.exists():
    # Función para extraer texto del PDF
    def extraer_texto_pdf(pdf_path,pagina_inicio=2):
        texto = ""
        with fitz.open(pdf_path) as pdf:
            for pagina in pdf:
                # Comenzar a extraer desde la página especificada
                if pagina.number >= pagina_inicio:
                    texto += pagina.get_text() + "\n"
                 #  texto += "\n"  # Agregar un salto de línea entre páginas

        texto = texto.replace("...", "")  # Eliminar puntos suspensivos
        # Limpiar texto
        texto = ' '.join(texto.split())  # Reemplazar múltiples espacios por uno solo
                # Filtrar texto para obtener solo contenido relevante
        lineas_relevantes = []
        for linea in texto.split('\n'):
            if re.search(r'\b(importante|relevante|objetivo|política)\b', linea, re.IGNORECASE):
                lineas_relevantes.append(linea)

        # Unir las líneas relevantes
        texto_relevante = '\n'.join(lineas_relevantes)
        # Estructurar el texto relevante
        secciones = re.split(r'(\d+\.\s)', texto_relevante)  # Separar por secciones numeradas
        texto_organizado = ""

        for i in range(1, len(secciones), 2):  # Iterar por las secciones
            encabezado = secciones[i].strip()
            contenido = secciones[i + 1].strip() if i + 1 < len(secciones) else ""
            texto_organizado += f"{encabezado} {contenido}\n\n"

        return texto_organizado

    # Extraer el texto
    texto_pdf = extraer_texto_pdf(pdf_path, pagina_inicio=2)
    print(texto_pdf)

    # Imprimir longitud del texto
    print(f"Longitud del texto extraído: {len(texto_pdf)} caracteres")
    # Imprimir los primeros 100 caracteres
    print(f"Primeros 100 caracteres:\n{texto_pdf[:100]}")
else:
    print("El archivo PDF no existe en la ruta especificada.")


In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
import re
import fitz

def limpiar_texto(texto):
    # Elimina caracteres especiales, dejando solo letras, números y espacios
    texto = re.sub(r'[...]+', '', texto)  # Eliminar puntos suspensivos
    texto = re.sub(r'(\d+)\s+(\d+)', r'\1.\2', texto)
    texto = re.sub(r'[^a-zA-Z0-9áéíóúñÑ.\s./]', '', texto)
    return ' '.join(texto.split())

texto = ""
# Abre el documento PDF y extrae texto
with fitz.open(pdf_path) as doc:
    for pagina in doc:
        texto += pagina.get_text() + "\n"


texto_limpio = limpiar_texto(texto)
print(f"Longitud del texto limpio: {len(texto_limpio)}")

documents = [Document(page_content=texto_limpio)]
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=30)

split_documents = text_splitter.split_documents(documents)#ok
#print(texto_limpio)
print(f"Longitud del texto original: {len(texto)}")
print(f"Número de chunks: {len(split_documents)}")

# Imprime los chunks
for i, chunk in enumerate(split_documents):
    print(f"Chunk {i}: {chunk.page_content}")


## Divide documento en partes (*chunks*)

Esto cumple dos funciones:

 * Mejorar la relevancia semántica de nuestros embeddings: un documento muy grande puede cubrir demasiados temas, mientras que uno pequeño puede estar más enfocado en un solo tópico
 * Facilitar la tarea del modelo de lenuaje generativo – *chunks* más pequeños hacen que la ventana de contexto sea más pequeña

El proceso de división tiene varios parámetros: el tamaño del *chunk* y el tamaño de traslape entre *chunks*.

In [ ]:
#pip install -U langchain-community

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

embeddings = HuggingFaceBgeEmbeddings()
query_result = embeddings.embed_query(split_documents[0].page_content)
query_result

## Guardamos a la base de datos tipo vectorial Chroma

In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(split_documents, embeddings)

# Verifica si el vectorstore se ha creado correctamente
print(f"Vectorstore creado con {len(vectorstore)} vectores.")

In [23]:
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFacePipeline
from transformers import pipeline  # Importa Hugging Face
from langchain_community.llms import Ollama

# Inicializar el modelo Llama 3
llm = Ollama(model="llama3", temperature=0.1)

qa = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(),
   # return_source_document = True
)

In [ ]:
chat_history = []
cache = {}
#query = "Por favor, responde únicamente en español: Dime los expedientes por categoria por aportaciones a Praxis"
query = "Por favor, responde únicamente en español: Dime los Expedientes por Categoría, que corresponde al documento Circulo de excelencia y compromiso PLRH-0313"

# Verificar si la consulta ya está en el caché
if query in cache:
    respuesta = cache[query]
    print("Respuesta desde caché:", respuesta)
else:
    # Si no está en el caché, hacer la consulta al modelo
    inputs = {
        "question": query,
        "chat_history": chat_history
    }
    print(inputs) 

    result = qa(inputs)
    print("Resultado completo:", result)  # Imprime el resultado completo para inspección
#result = qa({"Question": query, "chat_history": chat_history})
if "answer" in result:
    respuesta = result["answer"]
    print("Respuesta:", respuesta)
    # Actualizar el historial de chat
    chat_history.append((query, respuesta))
else:
    print("No se obtuvo respuesta.")

In [ ]:
# Mostrar el historial de chat
print("Historial de chat:")
for pregunta, respuesta in chat_history:
    print(f"Pregunta: {pregunta}\nRespuesta: {respuesta}\n")